In [1]:
from google.colab import auth
auth.authenticate_user()

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
from google.colab import files
uploaded = files.upload()

!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json


Saving kaggle.json to kaggle.json


In [4]:
!pip install accelerate torchcontrib -U >> /dev/null
!pip install timm colorama albumentations -U >> /dev/null
!pip install kaggle -U >> /dev/null

In [5]:
# ダウンロード
!kaggle datasets download nischaydnk/isic-2018-jpg-256x256-resized
!kaggle datasets download nischaydnk/isic-2019-jpg-256x256-resized
!kaggle datasets download nischaydnk/isic-2020-jpg-256x256-resized
# !kaggle competitions download -c isic-2024-challenge

# 解凍とzipファイルの削除
!unzip /content/isic-2018-jpg-256x256-resized.zip -d /content/isic-2018-jpg-256x256-resized >> /dev/null
!unzip /content/isic-2019-jpg-256x256-resized.zip -d /content/isic-2019-jpg-256x256-resized >> /dev/null
!unzip /content/isic-2020-jpg-256x256-resized.zip -d /content/isic-2020-jpg-256x256-resized >> /dev/null
# !unzip /content/isic-2024-challenge.zip -d /content/isic-2024-challenge && rm /content/isic-2024-challenge.zip


Dataset URL: https://www.kaggle.com/datasets/nischaydnk/isic-2018-jpg-256x256-resized
License(s): CC0-1.0
isic-2018-jpg-256x256-resized.zip: Skipping, found more recently modified local copy (use --force to force download)
Dataset URL: https://www.kaggle.com/datasets/nischaydnk/isic-2019-jpg-256x256-resized
License(s): CC0-1.0
isic-2019-jpg-256x256-resized.zip: Skipping, found more recently modified local copy (use --force to force download)
Dataset URL: https://www.kaggle.com/datasets/nischaydnk/isic-2020-jpg-256x256-resized
License(s): CC0-1.0
isic-2020-jpg-256x256-resized.zip: Skipping, found more recently modified local copy (use --force to force download)
replace /content/isic-2018-jpg-256x256-resized/.ipynb_checkpoints/dataset-metadata-checkpoint.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: replace /content/isic-2019-jpg-256x256-resized/.ipynb_checkpoints/dataset-metadata-checkpoint.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: replace /content/isic-2020-jpg-256x256-resized/.ipynb_c

In [6]:
import pandas as pd

metadata_2018 = pd.read_csv(f"/content/isic-2018-jpg-256x256-resized/train-metadata.csv")
metadata_2019 = pd.read_csv(f"/content/isic-2019-jpg-256x256-resized/train-metadata.csv")
metadata_2020 = pd.read_csv(f"/content/isic-2020-jpg-256x256-resized/train-metadata.csv")
# metadata_2024 = pd.read_csv(f"{COMMON_DIR}/isic-2024-challenge/train-metadata.csv")[['isic_id', 'patient_id', 'target']]

metadata_2018["img_size"] = 256
metadata_2018["year"] = 2018
metadata_2018["file_path"] = metadata_2018["isic_id"].apply(lambda x: f"/content/isic-2018-jpg-256x256-resized/train-image/image/" + x + ".jpg")
metadata_2019["img_size"] = 256
metadata_2019["year"] = 2019
metadata_2019["file_path"] = metadata_2019["isic_id"].apply(lambda x: f"/content/isic-2019-jpg-256x256-resized/train-image/image/" + x + ".jpg")
metadata_2020["img_size"] = 256
metadata_2020["year"] = 2020
metadata_2020["file_path"] = metadata_2020["isic_id"].apply(lambda x: f"/content/isic-2020-jpg-256x256-resized/train-image/image/" + x + ".jpg")
# metadata_2024["size"] = 256 # 一旦256を入れておく
# metadata_2024["year"] = 2024
# metadata_2024["file_path"] = metadata_2024["isic_id"].apply(lambda x: f"{COMMON_DIR}/isic-2024-challenge/train-image/image/" + x + ".jpg")

metadata_df = pd.concat([metadata_2018, metadata_2019, metadata_2020])
metadata_df = metadata_df[['isic_id', 'patient_id', 'target', 'img_size', 'year', 'file_path']]
metadata_df.head()

past_img_df = metadata_df[metadata_df['year'] <= 2020].reset_index(drop=True)

from sklearn.model_selection import StratifiedKFold

# TODO 将来的にはvalidation dataに2024のものを利用する方が良いかも?
for seed in [7, 42, 1000]:
    n_splits = 5
    past_img_df[f'skf_fold_num_5_seed_{seed}'] = -1

    # StratifiedKFold
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=seed)

    for fold, (_, val_idx) in enumerate(skf.split(past_img_df, past_img_df['target'])):
        past_img_df.loc[val_idx, f'skf_fold_num_5_seed_{seed}'] = fold

display(past_img_df.head())


,isic_id,patient_id,target,img_size,year,file_path,skf_fold_num_5_seed_7,skf_fold_num_5_seed_42,skf_fold_num_5_seed_1000
0,ISIC_0028498,HAM_0000000,0.0,256,2018,/content/isic-2018-jpg-256x256-resized/train-i...,2,2,4
1,ISIC_0027859,HAM_0000001,0.0,256,2018,/content/isic-2018-jpg-256x256-resized/train-i...,2,0,0
2,ISIC_0034246,HAM_0000002,1.0,256,2018,/content/isic-2018-jpg-256x256-resized/train-i...,4,2,1
3,ISIC_0027886,HAM_0000003,0.0,256,2018,/content/isic-2018-jpg-256x256-resized/train-i...,4,0,2
4,ISIC_0024645,HAM_0000004,0.0,256,2018,/content/isic-2018-jpg-256x256-resized/train-i...,3,0,4


In [7]:
import os
# os.environ["CUDA_VISIBLE_DEVICES"]="0,1"

COMMON_DIR="/content/drive/MyDrive/kaggle/isic2024"
ONLY_ONE_FOLD = True
TEST_RUN = False
TARGET_EXP_VERS = [10]


In [8]:
import os
import gc
import cv2
import math
import copy
import time
import random
import glob
from matplotlib import pyplot as plt

# For data manipulation
import numpy as np
import pandas as pd

# Pytorch Imports
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
from torch.cuda import amp
import torchvision

# Utils
import joblib
from tqdm import tqdm
from collections import defaultdict

# Sklearn Imports
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, StratifiedGroupKFold

# For Image Models
import timm

# Albumentations for augmentations
import albumentations as A
from albumentations.pytorch import ToTensorV2

# For colored terminal text
from colorama import Fore, Back, Style
b_ = Fore.BLUE
sr_ = Style.RESET_ALL

import warnings
warnings.filterwarnings("ignore")

# For descriptive error messages
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"


In [9]:
# GSSからハイパラ設定等を読み込んでCFGクラスに登録する
import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe
from google.auth import default
creds, _ = default()
gc = gspread.authorize(creds)

url = "https://docs.google.com/spreadsheets/d/1FoTSNZKKb-T7zxGz7FVUrpfSiTyo1-Ix0XPR-ORbWsY/edit?gid=0#gid=0"
ss = gc.open_by_url(url)
param_sheet = ss.worksheet("パラメータv1")
df_param = get_as_dataframe(param_sheet)


In [10]:
from pydantic import BaseModel
from typing import Optional

# default configs
class CriterionCFG(BaseModel):
    criterion_type: str = "FocalLoss"
    focal_gamma: Optional[float] = 2.0


class ModelCFG(BaseModel):
    backbone_type: str = "microsoft/deberta-v3-large"
    gradient_checkpointing: bool = False
    pretrained: bool = True
    pooling_type: str = "MeanPooling"
    freeze_n_layers: Optional[int] = 0
    reinitialize_n_layers: int = 0
    criterion: Optional[CriterionCFG] = None


class OptimizerCFG(BaseModel):
    encoder_lr: Optional[float] = 1e-5
    decoder_lr: Optional[float] = 1e-5
    embeddings_lr: Optional[float] = 1e-5
    weight_decay: Optional[float] = 0.01
    eps: Optional[float] = 1e-8
    betas: Optional[tuple] = (0.9, 0.999)
    use_swa: Optional[bool] = False


class SchedulerCFG(BaseModel):
    scheduler_type: str = "linear"
    num_warmup_steps: Optional[int] = 0
    num_cycles: Optional[float] = 0.5


class CFG(BaseModel):
    exp_ver: Optional[str] = "v2"
    n_splits: Optional[int] = 5
    seed: Optional[int] = 42
    lr: Optional[float] = 1e-5
    gradient_accumulation_steps: Optional[int] = 4
    save_eval_step_base: Optional[int] = 10 # 小さいほどこまめにcheck pointを保存
    train_batch_size: Optional[int] = 256
    eval_batch_size: Optional[int] = 512
    train_epochs: Optional[int] = 3
    weight_decay: Optional[float] = 0.01
    warmup_ratio: Optional[float] = 0.0
    num_labels: Optional[int] = 1
    fold_type: str = "stratified"
    freeze_n_layers: Optional[int] = 6
    scheduler_type: Optional[str] = "linear"
    user_persuade_data: Optional[int] = 0
    pool_type: Optional[str] = "GeM"
    criterion_type: Optional[str] = "FocalLoss"
    weight_decay: Optional[float] = 0.01
    criterion: Optional[CriterionCFG] = None
    optimizer: Optional[OptimizerCFG] = None
    scheduler: Optional[SchedulerCFG] = None
    model: Optional[ModelCFG] = None
    tokenizer_emb_len: Optional[int] = None
    use_prompt: Optional[int] = 0
    use_task_token: Optional[int] = 0
    save_pseudo_label: Optional[int] = 0
    img_size: int = 256


# configs from GSS
all_config_list = []
for exp_ver in TARGET_EXP_VERS:
    cfg = CFG()
    i = exp_ver - 1
    cfg.exp_ver = int(exp_ver)
    cfg.seed = int(df_param.loc[i, "seed"])
    cfg.gradient_accumulation_steps = int(df_param.loc[i, "gradient_accumulation_steps"])
    cfg.train_batch_size = int(df_param.loc[i, "train_batch_size"])
    cfg.eval_batch_size = int(df_param.loc[i, "eval_batch_size"])
    cfg.train_epochs = int(df_param.loc[i, "train_epochs"])
    cfg.fold_type = str(df_param.loc[i, "fold_type"])
    cfg.freeze_n_layers = int(df_param.loc[i, "freeze_n_layers"])
    cfg.scheduler_type = str(df_param.loc[i, "scheduler_type"])
    cfg.user_persuade_data = int(df_param.loc[i, "user_persuade_data"])
    cfg.criterion_type = str(df_param.loc[i, "criterion_type"])
    cfg.weight_decay = float(df_param.loc[i, "weight_decay"])
    cfg.img_size = int(df_param.loc[i, "img_size"])

    criterion_cfg = CriterionCFG()
    criterion_cfg.criterion_type = str(df_param.loc[i, "criterion_type"])
    criterion_cfg.focal_gamma = float(df_param.loc[i, "focal_gamma"])
    cfg.criterion = criterion_cfg

    model_cfg = ModelCFG()
    model_cfg.backbone_type = str(df_param.loc[i, "backbone_type"])
    model_cfg.pooling_type = str(df_param.loc[i, "pooling_type"])
    model_cfg.freeze_n_layers = int(df_param.loc[i, "freeze_n_layers"])
    model_cfg.reinitialize_n_layers = int(df_param.loc[i, "reinitialize_n_layers"])
    cfg.model = model_cfg

    optm_cfg = OptimizerCFG()
    optm_cfg.encoder_lr = float(df_param.loc[i, "encoder_lr"])
    optm_cfg.decoder_lr = float(df_param.loc[i, "decoder_lr"])
    optm_cfg.embeddings_lr = float(df_param.loc[i, "embeddings_lr"])
    optm_cfg.weight_decay = float(df_param.loc[i, "weight_decay"])
    optm_cfg.eps = float(df_param.loc[i, "eps"])
    optm_cfg.betas = (0.9, 0.999)  # デフォルト値の設定
    cfg.optimizer = optm_cfg

    scheduler_cfg = SchedulerCFG()
    scheduler_cfg.scheduler_type = str(df_param.loc[i, "scheduler_type"])
    scheduler_cfg.num_warmup_steps = 0  # デフォルト値の設定
    scheduler_cfg.num_cycles = 0.5  # デフォルト値の設定
    cfg.scheduler = scheduler_cfg

    all_config_list.append(cfg)


In [11]:
from pathlib import Path

data_path = Path(COMMON_DIR)

# TODO metadataを追加していく
# (past_metadataという変数名をつけているがimgへのpathがメインで格納されているだけなのでpast_img_pathとrenameする)
class PATHS:
    past_img_path = data_path / "past_img_df.csv"


In [12]:
def get_img_transform(CFG, is_train=False):
    if is_train:
        return A.Compose([
            A.Resize(CFG.img_size, CFG.img_size),
            A.RandomRotate90(p=0.5),
            A.Flip(p=0.5),
            A.Downscale(p=0.25),
            A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.15, rotate_limit=60, p=0.5),
            A.HueSaturationValue(hue_shift_limit=0.2, sat_shift_limit=0.2, val_shift_limit=0.2, p=0.5),
            A.RandomBrightnessContrast(brightness_limit=(-0.1,0.1), contrast_limit=(-0.1, 0.1), p=0.5),
            A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
            ToTensorV2()
        ], p=1.)
    else:
        return A.Compose([
            A.Resize(CFG.img_size, CFG.img_size),
            A.Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225],
                max_pixel_value=255.0,
                p=1.0
            ),
            ToTensorV2()], p=1.)

# TODO 試してみる
# 2020 1st Place Solution Transform Strategy
def get_img_transform(CFG, is_train=False):
    if is_train:
        return A.Compose([
            A.Resize(CFG.img_size, CFG.img_size),
            A.RandomRotate90(p=0.5),
            A.Flip(p=0.5),
            A.Downscale(p=0.25),
            A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.15, rotate_limit=60, p=0.5),
            A.HueSaturationValue(hue_shift_limit=0.2, sat_shift_limit=0.2, val_shift_limit=0.2, p=0.5),
            A.RandomBrightnessContrast(brightness_limit=(-0.1,0.1), contrast_limit=(-0.1, 0.1), p=0.5),
            A.OneOf([
                A.MotionBlur(blur_limit=5),
                A.MedianBlur(blur_limit=5),
                A.GaussianBlur(blur_limit=5),
                A.GaussNoise(var_limit=(5.0, 30.0)),
            ], p=0.7),
            A.OneOf([
                A.OpticalDistortion(distort_limit=1.0),
                A.GridDistortion(num_steps=5, distort_limit=1.),
                A.ElasticTransform(alpha=3),
            ], p=0.7),
            A.CLAHE(clip_limit=4.0, p=0.7),
            A.HueSaturationValue(hue_shift_limit=10, sat_shift_limit=20, val_shift_limit=10, p=0.5),
            A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=15, border_mode=0, p=0.85),
            # A.Cutout(max_h_size=int(CFG.img_size * 0.375), max_w_size=int(CFG.img_size * 0.375), num_holes=1, p=0.7),
            # A.CoarseDropout(max_holes=1, max_height=int(CFG.img_size * 0.375), max_width=int(CFG.img_size * 0.375), p=0.7),
            A.CoarseDropout(max_holes=1, max_height=int(CFG.img_size * 0.25), max_width=int(CFG.img_size * 0.25), p=0.5),
            A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
            ToTensorV2()
        ], p=1.)
    else:
        return A.Compose([
            A.Resize(CFG.img_size, CFG.img_size),
            A.Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225],
                max_pixel_value=255.0,
                p=1.0
            ),
            ToTensorV2()
        ], p=1.)


In [13]:
import random
import cv2
from torch.utils.data import Dataset


# TODO metadata特徴量を追加する
class ISICDataset(Dataset):
    def __init__(self, df, is_train=False, transforms=None):
        self.is_train = is_train
        self.df = df.reset_index(drop=True)
        self.file_names = df['file_path'].values
        self.targets = df['target'].values
        self.transforms = transforms

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        img_path = self.file_names[index]
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        target = self.targets[index]

        if self.transforms:
            img = self.transforms(image=img)["image"]

        return {
            'image': img,
            'target': target
        }

import numpy as np
import cv2
from torch.utils.data import Dataset

# class ISICDataset(Dataset):
#     def __init__(self, df, is_train=False, transforms=None):
#         self.is_train = is_train
#         self.df = df.reset_index(drop=True)
#         self.file_names = df['file_path'].values
#         self.targets = df['target'].values
#         self.transforms = transforms

#         # メタデータ特徴量を追加
#         # self.metadata = df[['meta1', 'meta2', 'meta3']].values  # 例として 'meta1', 'meta2', 'meta3' を使用

#         # target に基づいてデータを分割
#         self.df_target1 = df[df['target'] == 1].reset_index(drop=True)
#         self.df_target0 = df[df['target'] == 0].reset_index(drop=True)

#         # インデックスカウンタの初期化
#         self.index1 = 0
#         self.index0 = 0

#         # DataLoaderのshuffleの影響を避けるためのリスト
#         self.indices1 = np.arange(len(self.df_target1))
#         self.indices0 = np.arange(len(self.df_target0))

#     def __len__(self):
#         return len(self.df)

#     def __getitem__(self, index):
#         if self.is_train:
#             # 確率に基づいて target を選択
#             if np.random.rand() < 0.4:
#                 if self.index1 >= len(self.indices1):
#                     np.random.shuffle(self.indices1)
#                     self.index1 = 0
#                 selected_df = self.df_target1
#                 selected_index = self.indices1[self.index1]
#                 self.index1 += 1
#             else:
#                 if self.index0 >= len(self.indices0):
#                     np.random.shuffle(self.indices0)
#                     self.index0 = 0
#                 selected_df = self.df_target0
#                 selected_index = self.indices0[self.index0]
#                 self.index0 += 1
#         else:
#             selected_df = self.df
#             selected_index = index

#         img_path = selected_df['file_path'].values[selected_index]
#         img = cv2.imread(img_path)
#         img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
#         target = selected_df['target'].values[selected_index]

#         if self.transforms:
#             img = self.transforms(image=img)["image"]

#         # メタデータ特徴量を取得
#         # metadata = selected_df[['meta1', 'meta2', 'meta3']].values[selected_index]

#         return {
#             'image': img,
#             'target': target,
#             # 'metadata': metadata  # メタデータを返却
#         }


In [14]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.parameter import Parameter
import numpy as np


class GeM(nn.Module):
    def __init__(self, p=3, eps=1e-6):
        super(GeM, self).__init__()
        self.p = nn.Parameter(torch.ones(1)*p)
        self.eps = eps

    def forward(self, x):
        return self.gem(x, p=self.p, eps=self.eps)

    def gem(self, x, p=3, eps=1e-6):
        return F.avg_pool2d(x.clamp(min=eps).pow(p), (x.size(-2), x.size(-1))).pow(1./p)

    def __repr__(self):
        return self.__class__.__name__ + \
                '(' + 'p=' + '{:.4f}'.format(self.p.data.tolist()[0]) + \
                ', ' + 'eps=' + str(self.eps) + ')'


# TODO Pooling Strategyを追加していく
def get_pooling_layer(pooling_type: str = "GeM"):
    if pooling_type == 'GeM':
        return GeM()
    elif pooling_type == 'AdaptiveAvgPool2d':
        return nn.AdaptiveAvgPool2d(1)
    elif pooling_type == 'AdaptiveMaxPool2d':
        return nn.AdaptiveMaxPool2d(1)
    else:
        raise ValueError(f'Invalid pooling type: {pooling_type}')


def freeze(module):
    for parameter in module.parameters():
        parameter.requires_grad = False


# TODO sigmoid通す必要があるかを再度検討
import torch.nn as nn
import timm

def get_in_features(backbone):
    # backboneの構造を確認して、適切な層からin_featuresを取得する
    if hasattr(backbone, 'head') and hasattr(backbone.head, 'in_features'):
        print(backbone.head.in_features)
        return backbone.head.in_features
    elif hasattr(backbone, 'fc') and hasattr(backbone.fc, 'in_features'):
        print(backbone.fc.in_features)
        return backbone.fc.in_features
    elif hasattr(backbone, 'classifier') and hasattr(backbone.classifier, 'in_features'):
        print(backbone.classifier.in_features)
        return backbone.classifier.in_features
    elif hasattr(backbone, 'blocks') and hasattr(backbone.blocks[-1], 'mlp') and hasattr(backbone.blocks[-1].mlp, 'fc2'):
        print(backbone.blocks[-1].mlp.fc2.out_features)
        return backbone.blocks[-1].mlp.fc2.out_features
    else:
        raise AttributeError("Cannot find in_features in backbone")

class ISICModel(nn.Module):
    def __init__(
        self,
        backbone_type: str,
        pooling_type: str = "GeM",
        num_classes=1,
        pretrained=True,
        checkpoint_path=None,
        use_metadata=False,
    ):
        super(ISICModel, self).__init__()
        self.backbone_type = backbone_type
        self.backbone = timm.create_model(
            backbone_type,
            pretrained=pretrained,
            checkpoint_path=checkpoint_path,
        )

        in_features = get_in_features(self.backbone)
        if "resnext" in self.backbone_type:
            in_features = 1000

        # 必要に応じてheadを置き換える
        if hasattr(self.backbone, 'head'):
            self.backbone.head = nn.Identity()
        if hasattr(self.backbone, 'global_pool'):
            if "vit" in self.backbone_type:
                self.backbone.global_pool = None  # ここを修正
            if "resnext" in self.backbone_type:
               pass
            else:
                self.backbone.global_pool = nn.Identity()


        self.pooling = get_pooling_layer(pooling_type)
        self.linear = nn.Linear(in_features, num_classes)

    # input_ids, attention_mask, token_type_idsはTrainerを使用するためのdummy
    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        images=None,
        labels=None,
        **kwargs,
    ):
        features = self.backbone(images)

        if "swin" in self.backbone_type:
            features = features.permute(0, 3, 1, 2)  # [batch_size, hidden_dim, 7, 7]
            pooled_features = self.pooling(features).view(features.size(0), -1)  # [batch_size, hidden_dim]
            logits = self.linear(pooled_features)
        elif "resnext" in self.backbone_type:
            logits = self.linear(features).flatten(1)
        else:
            pooled_features = self.pooling(features).flatten(1)
            logits = self.linear(pooled_features)

        return {"logits": logits}


In [15]:
import torch
import torch.nn as nn
import torch.nn.functional as F


# TODO Custom lossを追加していく
class BinaryFocalLoss(torch.nn.Module):
    def __init__(self, weight: torch.Tensor | None = None, gamma: float = 2, reduction: str = 'mean') -> None:
        super().__init__()
        self.gamma = gamma
        self.weight = weight
        self.reduction = reduction

    def forward(self, input: torch.Tensor, target: torch.Tensor) -> torch.Tensor:
        bce_loss = F.binary_cross_entropy_with_logits(input, target, weight=self.weight, reduction='none')
        pt = torch.exp(-bce_loss)
        focal_loss = (1 - pt) ** self.gamma * bce_loss

        if self.reduction == 'mean':
            return torch.mean(focal_loss)
        elif self.reduction == 'sum':
            return torch.sum(focal_loss)
        else:
            return focal_loss


def get_criterion(config):
    if config.criterion.criterion_type == 'FocalLoss':
        return BinaryFocalLoss(gamma=2.0)
    # else:
    #     raise ValueError(f"Unsupported loss function: {config.criterion.criterion_type}")

    return nn.BCEWithLogitsLoss()


In [16]:
import math
from transformers import (
    get_linear_schedule_with_warmup,
    get_cosine_schedule_with_warmup,
    get_polynomial_decay_schedule_with_warmup,
    get_constant_schedule_with_warmup
)
from torch.optim import AdamW
from torchcontrib.optim import SWA


def get_optimizer_params(model, encoder_lr, decoder_lr, weight_decay=0.0):
    no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
    optimizer_parameters = [
        {
            'params': [p for n, p in model.backbone.named_parameters() if not any(nd in n for nd in no_decay)],
            'lr': encoder_lr,
            'weight_decay': weight_decay
        },
        {
            'params': [p for n, p in model.backbone.named_parameters() if any(nd in n for nd in no_decay)],
            'lr': encoder_lr,
            'weight_decay': 0.0
        },
        {
            'params': [p for n, p in model.named_parameters() if "backbone" not in n],
            'lr': decoder_lr,
            'weight_decay': 0.0
        }
    ]
    return optimizer_parameters


def get_optimizer(model, config):
    optimizer_parameters = get_optimizer_params(
        model,
        config.optimizer.encoder_lr,
        config.optimizer.decoder_lr,
        weight_decay=config.optimizer.weight_decay
    )
    optimizer = AdamW(
        optimizer_parameters,
        lr=config.optimizer.encoder_lr,
        eps=config.optimizer.eps,
        betas=config.optimizer.betas
    )

    if config.optimizer.use_swa:
        optimizer = SWA(
            optimizer,
            swa_start=config.optimizer.swa.swa_start,
            swa_freq=config.optimizer.swa.swa_freq,
            swa_lr=config.optimizer.swa.swa_lr
        )

    return optimizer

def get_scheduler(optimizer, config, num_train_steps):
    scheduler_type = config.scheduler.scheduler_type

    if scheduler_type == 'constant_schedule_with_warmup':
        scheduler = get_constant_schedule_with_warmup(
            optimizer,
            # num_warmup_steps=config.scheduler.constant_schedule_with_warmup.n_warmup_steps
            num_warmup_steps=0
        )
    elif scheduler_type == 'linear_schedule_with_warmup':
        scheduler = get_linear_schedule_with_warmup(
            optimizer,
            # num_warmup_steps=config.scheduler.linear_schedule_with_warmup.n_warmup_steps,
            num_warmup_steps=0,
            num_training_steps=num_train_steps
        )
    elif scheduler_type == 'cosine_schedule_with_warmup':
        scheduler = get_cosine_schedule_with_warmup(
            optimizer,
            # num_warmup_steps=config.scheduler.cosine_schedule_with_warmup.n_warmup_steps,
            num_warmup_steps=0,
            # num_cycles=config.scheduler.cosine_schedule_with_warmup.n_cycles,
            num_cycles=0.5,
            num_training_steps=num_train_steps,
        )
    elif scheduler_type == 'polynomial_decay_schedule_with_warmup':
        scheduler = get_polynomial_decay_schedule_with_warmup(
            optimizer,
            # num_warmup_steps=config.scheduler.polynomial_decay_schedule_with_warmup.n_warmup_steps,
            num_warmup_steps=0,
            num_training_steps=num_train_steps,
            power=config.scheduler.polynomial_decay_schedule_with_warmup.power,
            lr_end=config.scheduler.polynomial_decay_schedule_with_warmup.min_lr
        )
    else:
        raise ValueError(f'Unknown scheduler: {scheduler_type}')

    return scheduler


In [17]:
import numpy as np
from sklearn.metrics import roc_curve, auc, roc_auc_score


def calculate_pAUC(y_true: pd.DataFrame, y_pred: pd.DataFrame, min_tpr: float=0.80) -> float:
    v_gt = abs(np.asarray(y_true)-1)

    # flip the submissions to their compliments
    v_pred = -1.0*np.asarray(y_pred)

    max_fpr = abs(1-min_tpr)

    # using sklearn.metric functions: (1) roc_curve and (2) auc
    fpr, tpr, _ = roc_curve(v_gt, v_pred, sample_weight=None)
    if max_fpr is None or max_fpr == 1:
        return auc(fpr, tpr)
    if max_fpr <= 0 or max_fpr > 1:
        raise ValueError("Expected min_tpr in range [0, 1), got: %r" % min_tpr)

    # Add a single point at max_fpr by linear interpolation
    stop = np.searchsorted(fpr, max_fpr, "right")
    x_interp = [fpr[stop - 1], fpr[stop]]
    y_interp = [tpr[stop - 1], tpr[stop]]
    tpr = np.append(tpr[:stop], np.interp(max_fpr, x_interp, y_interp))
    fpr = np.append(fpr[:stop], max_fpr)
    partial_auc = auc(fpr, tpr)

    #     # Equivalent code that uses sklearn's roc_auc_score
    #     v_gt = abs(np.asarray(solution.values)-1)
    #     v_pred = np.array([1.0 - x for x in submission.values])
    #     max_fpr = abs(1-min_tpr)
    #     partial_auc_scaled = roc_auc_score(v_gt, v_pred, max_fpr=max_fpr)
    #     # change scale from [0.5, 1.0] to [0.5 * max_fpr**2, max_fpr]
    #     # https://math.stackexchange.com/questions/914823/shift-numbers-into-a-different-range
    #     partial_auc = 0.5 * max_fpr**2 + (max_fpr - 0.5 * max_fpr**2) / (1.0 - 0.5) * (partial_auc_scaled - 0.5)

    return(partial_auc)


def compute_metrics(eval_pred):
    y_scores, y_true  = eval_pred[0], eval_pred[1]
    return {'pAUC80': calculate_pAUC(y_true, y_scores)}


In [18]:
from transformers import Trainer, TrainingArguments


class CustomTrainer(Trainer):
    def __init__(self, cfg, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.cfg = cfg
        self.loss_fct = get_criterion(cfg)

    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        outputs = model(images=inputs.get("images"))
        logits = outputs.get("logits")

        loss = self.loss_fct(logits, labels)

        return (loss, outputs) if return_outputs else loss


def collate_fn(batch):
    images = []
    labels = []

    for item in batch:
        images.append(torch.tensor(item['image']).float())
        labels.append(item['target'])

    images = torch.stack(images)
    labels = torch.tensor(labels).unsqueeze(1).float()

    return {'images': images, 'labels': labels}



In [19]:
def seed_everything(seed):
    import random, os
    import numpy as np
    import torch

    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(seed=42)


# data = pd.read_csv(PATHS.past_img_path).sample(frac=1).reset_index(drop=True)
data = past_img_df.copy()
# ISIC2024_prep_v2.ipynbでのdownload & uploadがcolab上で不安定でimageが存在しない場合があるため
data['file_exists'] = data['file_path'].apply(lambda x: os.path.exists(x))
data = data[data['file_exists']].drop(columns=['file_exists'])
data["target"] = data["target"].astype('int32')



In [20]:
data.head()

,isic_id,patient_id,target,img_size,year,file_path,skf_fold_num_5_seed_7,skf_fold_num_5_seed_42,skf_fold_num_5_seed_1000
0,ISIC_0028498,HAM_0000000,0,256,2018,/content/isic-2018-jpg-256x256-resized/train-i...,2,2,4
1,ISIC_0027859,HAM_0000001,0,256,2018,/content/isic-2018-jpg-256x256-resized/train-i...,2,0,0
2,ISIC_0034246,HAM_0000002,1,256,2018,/content/isic-2018-jpg-256x256-resized/train-i...,4,2,1
3,ISIC_0027886,HAM_0000003,0,256,2018,/content/isic-2018-jpg-256x256-resized/train-i...,4,0,2
4,ISIC_0024645,HAM_0000004,0,256,2018,/content/isic-2018-jpg-256x256-resized/train-i...,3,0,4


In [ ]:
for CFG in all_config_list:
    save_eval_step = int(CFG.save_eval_step_base / CFG.gradient_accumulation_steps)
    if TEST_RUN:
        data = data.sample(200).reset_index(drop=True)
        save_eval_step = 5
    save_eval_step = 50
    print(save_eval_step)

    for fold in range(len(data[CFG.fold_type].unique())):
        if ONLY_ONE_FOLD and fold > 0:
            continue

        if not os.path.exists(f'{COMMON_DIR}/output/v{CFG.exp_ver}/fold_{fold}'):
            os.makedirs(f'{COMMON_DIR}/output/v{CFG.exp_ver}/fold_{fold}')

        training_args = TrainingArguments(
            # output_dir=f'output_{CFG.exp_ver}',
            logging_dir = f'{COMMON_DIR}/output/v{CFG.exp_ver}/fold_{fold}',
            output_dir = f'{COMMON_DIR}/output/v{CFG.exp_ver}/fold_{fold}',

            fp16=True,
            learning_rate=CFG.lr,
            gradient_accumulation_steps=CFG.gradient_accumulation_steps,
            per_device_train_batch_size=CFG.train_batch_size,
            per_device_eval_batch_size=CFG.eval_batch_size,
            num_train_epochs=CFG.train_epochs,
            weight_decay=CFG.weight_decay,

            metric_for_best_model='pAUC80',
            evaluation_strategy='steps',
            eval_steps=save_eval_step,
            save_strategy='steps',
            save_steps=save_eval_step,
            save_total_limit=2,
            load_best_model_at_end=True,

            report_to='none',
            logging_first_step=True,

            remove_unused_columns=False,
        )

        train = data[data[CFG.fold_type] != fold]
        valid = data[data[CFG.fold_type] == fold].copy()

        model = ISICModel(
            backbone_type=CFG.model.backbone_type,
            pooling_type=CFG.model.pooling_type,
            num_classes=CFG.num_labels,
            pretrained=CFG.model.pretrained,
            checkpoint_path=None,
            # checkpoint_path=CFG.model.checkpoint_path,
            use_metadata=False,
            # use_metadata=CFG.use_metadata,
        )

        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        model.to(device)

        train_steps_per_epoch = int(len(train) / CFG.train_batch_size)
        num_train_steps = train_steps_per_epoch * CFG.train_epochs

        optimizer = get_optimizer(model, CFG)
        scheduler = get_scheduler(optimizer, CFG, num_train_steps)

        train_dataset = ISICDataset(train, is_train=True, transforms=get_img_transform(CFG, is_train=True))
        valid_dataset = ISICDataset(valid, is_train=False, transforms=get_img_transform(CFG, is_train=False))

        trainer = CustomTrainer(
            cfg=CFG,
            model=model,
            args=training_args,
            train_dataset=train_dataset,
            eval_dataset=valid_dataset,
            data_collator=collate_fn,
            compute_metrics=compute_metrics,
            optimizers=(optimizer, scheduler),
        )

        try:
            trainer.train()
        except Exception as e:
            print(e)
            # from google.colab import runtime
            # runtime.unassign()

        y_true = valid['target'].values
        tmp_preds = trainer.predict(valid_dataset).predictions

        try:
            partial_auc = calculate_pAUC(y_true, tmp_preds)
            param_sheet = ss.worksheet("パラメータv1")
            param_sheet.update_acell(f"Z{CFG.exp_ver + 1}", partial_auc)
        except Exception as e:
            print(e)

        best_model = trainer.model
        torch.save(
            {'model': model.state_dict()},
            f'{COMMON_DIR}/output/v{CFG.exp_ver}/fold_{fold}/best_model.pth',
        )

        valid['pred'] = tmp_preds
        valid.to_csv(f'{COMMON_DIR}/output/v{CFG.exp_ver}/valid_df_fold_{fold}.csv', index=False)

    # OOF Valid Score.
    if not ONLY_ONE_FOLD:
        dfs = []
        for k in range(CFG.n_splits):
            dfs.append( pd.read_csv(f'{COMMON_DIR}/output/v{CFG.exp_ver}/valid_df_fold_{k}.csv'))

        dfs = pd.concat(dfs)
        # Stacking用に保存しておく
        dfs.to_csv(f'{COMMON_DIR}/output/v{CFG.exp_ver}/valid_df.csv',index=False)
        print('Valid OOF shape:', dfs.shape )
        display( dfs.head() )

        m = calculate_pAUC(dfs.target.values, dfs['pred'].values)
        print('Overall pAUC80 CV =',m)

        try:
            param_sheet = ss.worksheet("パラメータv1")
            param_sheet.update_acell(f"W{CFG.exp_ver+1}", m)
        except Exception as e:
            print(e)


50
1024


Step,Training Loss,Validation Loss,Pauc80
50,0.504000,0.235681,0.114307
100,0.504000,0.224185,0.120353
150,0.504000,0.205229,0.132423
200,0.504000,0.206155,0.127939
250,0.504000,0.193183,0.133906
300,0.504000,0.194150,0.130885
350,0.504000,0.208985,0.131427
400,0.504000,0.186727,0.139514
450,0.504000,0.181907,0.141245
500,0.222600,0.188145,0.139523


Step,Training Loss,Validation Loss,Pauc80
50,0.504000,0.235681,0.114307
100,0.504000,0.224185,0.120353
150,0.504000,0.205229,0.132423
200,0.504000,0.206155,0.127939
250,0.504000,0.193183,0.133906
300,0.504000,0.194150,0.130885
350,0.504000,0.208985,0.131427
400,0.504000,0.186727,0.139514
450,0.504000,0.181907,0.141245
500,0.222600,0.188145,0.139523


In [ ]:
# from google.colab import runtime
# runtime.unassign()


In [ ]:
from google.colab import runtime
runtime.unassign()
